In [1]:
import itertools
import json
import os

import numpy as np
import scipy.sparse as sp

from src.data.Data_gen_utils import generate_UIO, is_P_compatible, is_P_less, P_Des, words_from_orbit, shape_of_word, \
    iter_shuffles, cluster_vertices, EDGE_TYPE, Direction, orbits_from_P, PTab_from_word
from src.data.criterion import *
from src.data.shapes import *

In [2]:
import itertools
import json
import os

import numpy as np
import scipy.sparse as sp

from src.data.Data_gen_utils import generate_UIO, is_P_compatible, is_P_less, P_Des, words_from_orbit, shape_of_word, \
    iter_shuffles, cluster_vertices, EDGE_TYPE, Direction, orbits_from_P, PTab_from_word
from src.data.criterion import *
from src.data.shapes import *
# === Step 1: Load Metadata ===
def load_metadata(json_path):
    with open(os.path.join(json_path, "Partitions.json")) as f:
        Partitions = json.load(f)
    with open(os.path.join(json_path, "PartitionIndex.json")) as f:
        PartitionIndex = json.load(f)
    with open(os.path.join(json_path, "TransitionMatrix.json")) as f:
        TM = json.load(f)
    return Partitions, PartitionIndex, TM

# === Step 2: Generate Words from Poset ===
def generate_words(P, N, primitive):
    if primitive:
        return iter_shuffles(cluster_vertices(P))
    else:
        return itertools.permutations(range(1, N + 1))

# === Step 3: Process Single Word ===
def process_word(P, word, shape_checkers, column_info):
    shape = shape_of_word(P, word)
    if shape is None:
        return None, None
    if not any(checker(shape) for checker in shape_checkers):
        return None, None

    if column_info == "original":
        g = make_matrix_from_T(P, word)
    elif column_info == "column_direction":
        g = make_matrix_from_T_col_info(P, word)
    elif column_info == "column_direc_column_same":
        g = make_matrix_from_T_col_info(P, word,
            direction=(Direction.FORWARD, Direction.BOTH, Direction.FORWARD))
    else:
        raise ValueError("Invalid column_info option")

    return shape, g

# === Step 4: Label Decision ===
def decide_label(P, word):
    chk = check_all_row_connected(P, word)
    if chk == "GOOD":
        return 1
    elif chk == "BAD":
        return 0
    elif chk == "UNKNOWN":
        return None
    else:
        raise ValueError("Unexpected chk result")

# === Step 5: Compute Schur Label ===
def compute_transition_label(TM_n, Fs, k):
    return sum(TM_n[i][k] * Fs[i] for i in range(len(Fs)))

# === Step 6: Save Graph and Metadata ===
def write_graph_and_metadata(g, label, size, DIR_PATH, idx):
    path = os.path.join(DIR_PATH, f"graph_{idx:05d}.npz")
    sp.save_npz(path, g)
    return {"index": idx, "label": label, "size": size}

# === Utility Functions ===
def load_metadata(json_path):
    with open(os.path.join(json_path, "Partitions.json")) as f:
        Partitions = json.load(f)
    with open(os.path.join(json_path, "PartitionIndex.json")) as f:
        PartitionIndex = json.load(f)
    with open(os.path.join(json_path, "TransitionMatrix_btw_s_h.json")) as f:
        TM = json.load(f)
    return Partitions, PartitionIndex, TM

def init_shape_counters(N, Partitions):
    gs, s_vec, pre_calc = {}, {}, {}
    for k in range(1, N+1):
        gs[k], s_vec[k], pre_calc[k] = {}, {}, {}
        for lamb in Partitions:
            lamb_str = str(lamb)
            gs[k][lamb_str] = sp.coo_matrix(([], ([], [])), shape=(0, 0), dtype=np.int16)
            s_vec[k][lamb_str] = 0
            pre_calc[k][lamb_str] = 0
    return gs, s_vec, pre_calc

def generate_graph_from_word(P, word, shape_checkers, column_info):
    shape = shape_of_word(P, word)
    if shape is None or not any(checker(shape) for checker in shape_checkers):
        return None, None
    if column_info == "original":
        g = make_matrix_from_T(P, word)
    elif column_info == "column_direction":
        g = make_matrix_from_T_col_info(P, word)
    elif column_info == "column_direc_column_same":
        g = make_matrix_from_T_col_info(P, word, direction=(Direction.FORWARD, Direction.BOTH, Direction.FORWARD))
    else:
        raise ValueError("Invalid column_info")
    return shape, g

def filter_word_and_calculate_scoeff(P, word, shape_checkers, column_info, filter_fn,
                                     gs, pre_calc, s_vec, saver, N, mode):
    shape, g = generate_graph_from_word(P, word, shape_checkers, column_info)
    if shape is None:
        return
    T = PTab_from_word(P, word)
    if mode == "decomp":
        k = T[0][0]
    elif mode == "position_one":
        k = T[0].index(1) + 1
    elif mode == "vanilla" or mode == "ppath" or mode == "test":
        k = 1
    else: 
        raise ValueError
    shape_str = str(shape)
    s_vec[k][shape_str] += 1
    result = filter_fn(P, word)
    if result == 'UNKNOWN':
        gs[k][shape_str] = sp.block_diag((gs[k][shape_str], g))
    elif result == 'BAD':
        saver.save_streaming(g, label=0, N=N)
    elif result == 'GOOD':
        saver.save_streaming(g, label=1, N=N)
        pre_calc[k][shape_str] += 1
    else:
        raise Exception(f"{P}, {word}, {k}, {shape_str}: filter error")
        
def calculate_hcoeff(TM_n, N, Partitions, gs, pre_calc, s_vec,
                     shape_checkers, P, word, saver):
    for k in s_vec.keys():
        for lamb in Partitions:
            lamb_str = str(lamb)
            if gs[k][lamb_str].size == 0 and pre_calc[k][lamb_str] == 0:
                continue
            if not any(chk(lamb) for chk in shape_checkers):
                continue
            h_coeff = 0
            for mu in Partitions:
                mu_str = str(mu)
                h_coeff += TM_n[mu_str][lamb_str] * s_vec[k][mu_str]
            if h_coeff < 0:
                raise Exception(f"{P}, {word}, {k}, {lamb}, {h_coeff}: negative coefficient!")
#             est_count = int(gs[k][lamb_str].size / Nchoose2)
            est_count = gs[k][lamb_str].shape[0] // N
#             if est_count != int(gs[k][lamb_str].size / Nchoose2):
#                 print(gs[k][lamb_str])
#                 print(est_count, int(gs[k][lamb_str].size / Nchoose2))
#                 raise ValueError
            usable = h_coeff - pre_calc[k][lamb_str]
            if est_count < usable:
                raise Exception(f"{P}, {word}, {k}, {lamb}, {h_coeff}: insufficient filtered data!")
            saver.save_streaming(gs[k][lamb_str], usable, N)


def process_orbits(P, orbit, N, Partitions, TM_n, shape_checkers,
                   column_info, filter_fn, saver, mode):
    gs, s_vec, pre_calc = init_shape_counters(N, Partitions)
    for word in orbit:
        filter_word_and_calculate_scoeff(P, word, shape_checkers, column_info, filter_fn,
                                         gs, pre_calc, s_vec, saver, N, mode)
    calculate_hcoeff(TM_n, N, Partitions,
                     gs, pre_calc, s_vec, shape_checkers, P, word, saver)
#     print(f"orbit with {orbit[0]} completed")

In [3]:
json_path = "src/data/json/"
Partitions, PartitionIndex, TM = load_metadata(json_path)
filter_fn = check_inductive_disconnectedness_criterion_forward

In [5]:
P = [2, 4, 4, 6, 6, 8, 8, 9, 10, 10]
word = [6, 10, 1, 5, 9, 7, 8, 3, 2, 4]

In [6]:
orbit = words_from_orbit(P,word)

In [7]:
_, s_vec, _ = init_shape_counters(10, Partitions['10'])

In [8]:
count = 0
for word in orbit:
    shape = shape_of_word(P, word)
    if shape:
        T = PTab_from_word(P, word)
        if T[0][0] == 3:
            print(f"word,{word}, shape , {shape}")
            print(f"T,{T}")
            s_vec[3][str(shape)]+=1
            count += 1

word,[6, 3, 2, 1, 5, 4, 8, 7, 9, 10], shape , [9, 1]
T,[[3, 2, 1, 5, 4, 8, 7, 9, 10], [6]]
word,[10, 8, 3, 9, 2, 1, 4, 5, 7, 6], shape , [7, 2, 1]
T,[[3, 2, 1, 4, 5, 7, 6], [8, 9], [10]]
word,[5, 3, 2, 1, 4, 8, 7, 6, 9, 10], shape , [9, 1]
T,[[3, 2, 1, 4, 8, 7, 6, 9, 10], [5]]
word,[7, 5, 3, 9, 6, 2, 10, 4, 1, 8], shape , [4, 3, 3]
T,[[3, 2, 1, 8], [5, 6, 4], [7, 9, 10]]
word,[3, 2, 1, 7, 6, 4, 5, 8, 10, 9], shape , [10]
T,[[3, 2, 1, 7, 6, 4, 5, 8, 10, 9]]
word,[9, 7, 5, 3, 6, 2, 4, 1, 8, 10], shape , [5, 3, 1, 1]
T,[[3, 2, 1, 8, 10], [5, 6, 4], [7], [9]]
word,[5, 3, 6, 2, 9, 1, 8, 4, 10, 7], shape , [5, 5]
T,[[3, 2, 1, 4, 7], [5, 6, 9, 8, 10]]
word,[8, 5, 3, 2, 1, 6, 4, 7, 9, 10], shape , [8, 1, 1]
T,[[3, 2, 1, 6, 4, 7, 9, 10], [5], [8]]
word,[5, 3, 7, 2, 9, 1, 8, 4, 10, 6], shape , [5, 5]
T,[[3, 2, 1, 4, 6], [5, 7, 9, 8, 10]]
word,[3, 2, 1, 4, 7, 9, 8, 6, 5, 10], shape , [10]
T,[[3, 2, 1, 4, 7, 9, 8, 6, 5, 10]]
word,[10, 6, 3, 4, 2, 1, 5, 7, 8, 9], shape , [8, 1, 1]
T,[[3, 4, 2, 1, 5

word,[8, 3, 4, 2, 1, 5, 7, 6, 9, 10], shape , [9, 1]
T,[[3, 4, 2, 1, 5, 7, 6, 9, 10], [8]]
word,[9, 3, 4, 2, 1, 6, 5, 8, 7, 10], shape , [9, 1]
T,[[3, 4, 2, 1, 6, 5, 8, 7, 10], [9]]
word,[9, 5, 3, 6, 2, 4, 1, 10, 8, 7], shape , [5, 4, 1]
T,[[3, 2, 1, 8, 7], [5, 6, 4, 10], [9]]
word,[5, 3, 10, 2, 1, 7, 6, 4, 8, 9], shape , [8, 2]
T,[[3, 2, 1, 7, 6, 4, 8, 9], [5, 10]]
word,[10, 6, 3, 7, 4, 2, 1, 5, 8, 9], shape , [7, 2, 1]
T,[[3, 4, 2, 1, 5, 8, 9], [6, 7], [10]]
word,[9, 7, 3, 8, 2, 1, 5, 4, 6, 10], shape , [7, 2, 1]
T,[[3, 2, 1, 5, 4, 6, 10], [7, 8], [9]]
word,[10, 5, 3, 7, 4, 2, 1, 6, 8, 9], shape , [7, 2, 1]
T,[[3, 4, 2, 1, 6, 8, 9], [5, 7], [10]]
word,[9, 6, 3, 2, 1, 5, 4, 7, 8, 10], shape , [8, 1, 1]
T,[[3, 2, 1, 5, 4, 7, 8, 10], [6], [9]]
word,[6, 3, 8, 4, 7, 2, 1, 5, 9, 10], shape , [7, 3]
T,[[3, 4, 2, 1, 5, 9, 10], [6, 8, 7]]
word,[8, 3, 10, 2, 9, 1, 5, 4, 6, 7], shape , [7, 3]
T,[[3, 2, 1, 5, 4, 6, 7], [8, 10, 9]]
word,[9, 6, 3, 7, 2, 1, 5, 4, 8, 10], shape , [7, 2, 1]
T,[[3, 2,

word,[5, 3, 2, 1, 7, 6, 4, 8, 10, 9], shape , [9, 1]
T,[[3, 2, 1, 7, 6, 4, 8, 10, 9], [5]]
word,[9, 5, 3, 7, 2, 10, 1, 4, 8, 6], shape , [6, 3, 1]
T,[[3, 2, 1, 4, 8, 6], [5, 7, 10], [9]]
word,[8, 5, 3, 9, 2, 10, 1, 4, 7, 6], shape , [6, 3, 1]
T,[[3, 2, 1, 4, 7, 6], [5, 9, 10], [8]]
word,[9, 5, 3, 10, 7, 4, 6, 2, 8, 1], shape , [4, 4, 2]
T,[[3, 4, 2, 1], [5, 7, 6, 8], [9, 10]]
word,[8, 3, 2, 1, 4, 7, 6, 5, 9, 10], shape , [9, 1]
T,[[3, 2, 1, 4, 7, 6, 5, 9, 10], [8]]
word,[10, 7, 3, 6, 2, 8, 1, 9, 4, 5], shape , [5, 4, 1]
T,[[3, 2, 1, 4, 5], [7, 6, 8, 9], [10]]
word,[7, 3, 8, 2, 1, 4, 6, 5, 10, 9], shape , [8, 2]
T,[[3, 2, 1, 4, 6, 5, 10, 9], [7, 8]]
word,[5, 3, 6, 2, 8, 1, 4, 7, 10, 9], shape , [7, 3]
T,[[3, 2, 1, 4, 7, 10, 9], [5, 6, 8]]
word,[3, 2, 1, 6, 4, 5, 9, 8, 7, 10], shape , [10]
T,[[3, 2, 1, 6, 4, 5, 9, 8, 7, 10]]
word,[5, 3, 7, 4, 6, 2, 1, 9, 8, 10], shape , [7, 3]
T,[[3, 4, 2, 1, 9, 8, 10], [5, 7, 6]]
word,[8, 3, 7, 2, 9, 1, 10, 4, 6, 5], shape , [6, 4]
T,[[3, 2, 1, 4, 6, 5]

word,[9, 6, 3, 10, 8, 2, 7, 1, 4, 5], shape , [5, 3, 2]
T,[[3, 2, 1, 4, 5], [6, 8, 7], [9, 10]]
word,[6, 3, 7, 2, 9, 1, 8, 5, 10, 4], shape , [5, 5]
T,[[3, 2, 1, 5, 4], [6, 7, 9, 8, 10]]
word,[10, 3, 9, 2, 1, 6, 4, 5, 7, 8], shape , [8, 2]
T,[[3, 2, 1, 6, 4, 5, 7, 8], [10, 9]]
word,[5, 3, 9, 4, 2, 1, 7, 6, 8, 10], shape , [8, 2]
T,[[3, 4, 2, 1, 7, 6, 8, 10], [5, 9]]
word,[5, 3, 8, 2, 9, 1, 4, 7, 6, 10], shape , [7, 3]
T,[[3, 2, 1, 4, 7, 6, 10], [5, 8, 9]]
word,[9, 6, 3, 5, 2, 7, 1, 4, 8, 10], shape , [6, 3, 1]
T,[[3, 2, 1, 4, 8, 10], [6, 5, 7], [9]]
word,[6, 3, 8, 4, 7, 2, 9, 1, 10, 5], shape , [5, 5]
T,[[3, 4, 2, 1, 5], [6, 8, 7, 9, 10]]
word,[9, 3, 6, 4, 2, 1, 5, 7, 8, 10], shape , [9, 1]
T,[[3, 6, 4, 2, 1, 5, 7, 8, 10], [9]]
word,[7, 3, 8, 4, 9, 2, 10, 1, 6, 5], shape , [6, 4]
T,[[3, 4, 2, 1, 6, 5], [7, 8, 9, 10]]
word,[10, 3, 2, 1, 4, 5, 8, 7, 6, 9], shape , [9, 1]
T,[[3, 2, 1, 4, 5, 8, 7, 6, 9], [10]]
word,[6, 3, 8, 2, 7, 1, 5, 4, 9, 10], shape , [7, 3]
T,[[3, 2, 1, 5, 4, 9, 10], 

word,[5, 3, 4, 2, 1, 7, 6, 9, 8, 10], shape , [9, 1]
T,[[3, 4, 2, 1, 7, 6, 9, 8, 10], [5]]
word,[5, 3, 6, 4, 2, 1, 8, 7, 9, 10], shape , [9, 1]
T,[[3, 6, 4, 2, 1, 8, 7, 9, 10], [5]]
word,[7, 3, 9, 2, 10, 1, 4, 8, 6, 5], shape , [7, 3]
T,[[3, 2, 1, 4, 8, 6, 5], [7, 9, 10]]
word,[10, 3, 9, 2, 1, 5, 4, 7, 6, 8], shape , [8, 2]
T,[[3, 2, 1, 5, 4, 7, 6, 8], [10, 9]]
word,[7, 5, 3, 8, 2, 9, 1, 6, 4, 10], shape , [6, 3, 1]
T,[[3, 2, 1, 6, 4, 10], [5, 8, 9], [7]]
word,[9, 7, 3, 10, 6, 2, 8, 1, 4, 5], shape , [5, 3, 2]
T,[[3, 2, 1, 4, 5], [7, 6, 8], [9, 10]]
word,[5, 3, 2, 1, 4, 8, 6, 7, 10, 9], shape , [9, 1]
T,[[3, 2, 1, 4, 8, 6, 7, 10, 9], [5]]
word,[6, 3, 5, 2, 7, 1, 9, 4, 10, 8], shape , [5, 5]
T,[[3, 2, 1, 4, 8], [6, 5, 7, 9, 10]]
word,[8, 5, 3, 4, 2, 1, 6, 7, 9, 10], shape , [8, 1, 1]
T,[[3, 4, 2, 1, 6, 7, 9, 10], [5], [8]]
word,[8, 3, 9, 2, 1, 5, 6, 4, 7, 10], shape , [8, 2]
T,[[3, 2, 1, 5, 6, 4, 7, 10], [8, 9]]
word,[9, 5, 3, 10, 4, 2, 1, 6, 8, 7], shape , [7, 2, 1]
T,[[3, 4, 2, 1, 6, 

word,[8, 3, 9, 2, 10, 1, 4, 7, 6, 5], shape , [7, 3]
T,[[3, 2, 1, 4, 7, 6, 5], [8, 9, 10]]
word,[7, 3, 8, 4, 2, 1, 5, 6, 10, 9], shape , [8, 2]
T,[[3, 4, 2, 1, 5, 6, 10, 9], [7, 8]]
word,[6, 3, 7, 2, 9, 1, 8, 5, 4, 10], shape , [6, 4]
T,[[3, 2, 1, 5, 4, 10], [6, 7, 9, 8]]
word,[6, 3, 4, 2, 1, 5, 7, 9, 8, 10], shape , [9, 1]
T,[[3, 4, 2, 1, 5, 7, 9, 8, 10], [6]]
word,[7, 3, 9, 2, 10, 1, 5, 4, 8, 6], shape , [7, 3]
T,[[3, 2, 1, 5, 4, 8, 6], [7, 9, 10]]
word,[7, 5, 3, 6, 4, 2, 1, 8, 9, 10], shape , [8, 1, 1]
T,[[3, 6, 4, 2, 1, 8, 9, 10], [5], [7]]
word,[6, 3, 8, 2, 10, 1, 4, 5, 7, 9], shape , [7, 3]
T,[[3, 2, 1, 4, 5, 7, 9], [6, 8, 10]]
word,[5, 3, 7, 4, 6, 2, 8, 1, 10, 9], shape , [6, 4]
T,[[3, 4, 2, 1, 10, 9], [5, 7, 6, 8]]
word,[10, 3, 2, 1, 5, 4, 7, 6, 9, 8], shape , [9, 1]
T,[[3, 2, 1, 5, 4, 7, 6, 9, 8], [10]]
word,[9, 6, 3, 7, 4, 8, 2, 1, 5, 10], shape , [6, 3, 1]
T,[[3, 4, 2, 1, 5, 10], [6, 7, 8], [9]]
word,[5, 3, 8, 2, 6, 1, 4, 7, 9, 10], shape , [7, 3]
T,[[3, 2, 1, 4, 7, 9, 10], 

word,[8, 3, 9, 4, 2, 1, 6, 5, 7, 10], shape , [8, 2]
T,[[3, 4, 2, 1, 6, 5, 7, 10], [8, 9]]
word,[7, 3, 10, 4, 2, 1, 5, 6, 9, 8], shape , [8, 2]
T,[[3, 4, 2, 1, 5, 6, 9, 8], [7, 10]]
word,[7, 3, 10, 2, 1, 4, 8, 6, 5, 9], shape , [8, 2]
T,[[3, 2, 1, 4, 8, 6, 5, 9], [7, 10]]
word,[10, 7, 3, 2, 1, 4, 8, 6, 5, 9], shape , [8, 1, 1]
T,[[3, 2, 1, 4, 8, 6, 5, 9], [7], [10]]
word,[10, 7, 3, 2, 1, 4, 5, 9, 8, 6], shape , [8, 1, 1]
T,[[3, 2, 1, 4, 5, 9, 8, 6], [7], [10]]
word,[7, 3, 5, 4, 2, 1, 6, 8, 10, 9], shape , [9, 1]
T,[[3, 5, 4, 2, 1, 6, 8, 10, 9], [7]]
word,[8, 3, 10, 2, 1, 5, 4, 6, 7, 9], shape , [8, 2]
T,[[3, 2, 1, 5, 4, 6, 7, 9], [8, 10]]
word,[10, 5, 3, 2, 1, 4, 8, 6, 7, 9], shape , [8, 1, 1]
T,[[3, 2, 1, 4, 8, 6, 7, 9], [5], [10]]
word,[5, 3, 6, 2, 8, 1, 10, 4, 9, 7], shape , [5, 5]
T,[[3, 2, 1, 4, 7], [5, 6, 8, 10, 9]]
word,[7, 5, 3, 9, 6, 2, 1, 4, 8, 10], shape , [6, 2, 2]
T,[[3, 2, 1, 4, 8, 10], [5, 6], [7, 9]]
word,[5, 3, 10, 2, 9, 1, 4, 7, 6, 8], shape , [7, 3]
T,[[3, 2, 1, 4, 7

word,[8, 3, 6, 2, 4, 1, 5, 7, 9, 10], shape , [7, 3]
T,[[3, 2, 1, 5, 7, 9, 10], [8, 6, 4]]
word,[5, 3, 7, 2, 10, 1, 6, 4, 8, 9], shape , [7, 3]
T,[[3, 2, 1, 6, 4, 8, 9], [5, 7, 10]]
word,[7, 3, 6, 2, 5, 1, 8, 4, 9, 10], shape , [6, 4]
T,[[3, 2, 1, 4, 9, 10], [7, 6, 5, 8]]
word,[9, 3, 10, 5, 4, 2, 1, 6, 8, 7], shape , [8, 2]
T,[[3, 5, 4, 2, 1, 6, 8, 7], [9, 10]]
word,[10, 7, 5, 3, 8, 2, 1, 4, 6, 9], shape , [6, 2, 1, 1]
T,[[3, 2, 1, 4, 6, 9], [5, 8], [7], [10]]
word,[10, 8, 3, 7, 2, 1, 4, 5, 6, 9], shape , [7, 2, 1]
T,[[3, 2, 1, 4, 5, 6, 9], [8, 7], [10]]
word,[6, 3, 7, 5, 8, 4, 2, 1, 9, 10], shape , [7, 3]
T,[[3, 5, 4, 2, 1, 9, 10], [6, 7, 8]]
word,[6, 3, 9, 2, 8, 1, 7, 4, 10, 5], shape , [5, 5]
T,[[3, 2, 1, 4, 5], [6, 9, 8, 7, 10]]
word,[9, 5, 3, 2, 1, 4, 8, 6, 7, 10], shape , [8, 1, 1]
T,[[3, 2, 1, 4, 8, 6, 7, 10], [5], [9]]
word,[5, 3, 8, 2, 9, 1, 10, 6, 4, 7], shape , [6, 4]
T,[[3, 2, 1, 6, 4, 7], [5, 8, 9, 10]]
word,[9, 6, 3, 7, 2, 10, 1, 5, 4, 8], shape , [6, 3, 1]
T,[[3, 2, 1, 5

In [9]:
count

909

In [10]:
h_coeff = 0
lamb_str = str([4,3,3])
for mu in Partitions['10']:
    mu_str = str(mu)
    h_coeff += TM['10'][mu_str][lamb_str] * s_vec[3][mu_str]

In [11]:
h_coeff

3

In [12]:
count = 0
for word in orbit:
    shape = shape_of_word(P, word)
    if shape == [4,3,3]:
        T = PTab_from_word(P, word)
        if T[0][0] == 3:
            result = filter_fn(P, word)
            print(f"word,{word}, shape , {shape}")
            print(f"T,{T}, result, {result}")
            s_vec[3][str(shape)]+=1
            count += 1

word,[7, 5, 3, 9, 6, 2, 10, 4, 1, 8], shape , [4, 3, 3]
T,[[3, 2, 1, 8], [5, 6, 4], [7, 9, 10]], result, BAD
word,[7, 5, 3, 9, 6, 2, 10, 8, 1, 4], shape , [4, 3, 3]
T,[[3, 2, 1, 4], [5, 6, 8], [7, 9, 10]], result, BAD
word,[8, 5, 3, 9, 6, 2, 10, 4, 1, 7], shape , [4, 3, 3]
T,[[3, 2, 1, 7], [5, 6, 4], [8, 9, 10]], result, UNKNOWN
word,[8, 5, 3, 9, 6, 2, 10, 7, 1, 4], shape , [4, 3, 3]
T,[[3, 2, 1, 4], [5, 6, 7], [8, 9, 10]], result, UNKNOWN
word,[7, 5, 3, 9, 6, 2, 8, 4, 1, 10], shape , [4, 3, 3]
T,[[3, 2, 1, 10], [5, 6, 4], [7, 9, 8]], result, BAD
